In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import os

In [204]:
from sqlite3 import connect

from collections import defaultdict
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_absolute_error

In [5]:
DATA_FOLDER = 'resources'
DATA_FILENAME = 'trade_info.sqlite3'

conn = connect(os.path.join(DATA_FOLDER, DATA_FILENAME))

In [10]:
sessions_table = pd.read_sql('SELECT * FROM Trading_session', conn)
sessions_table.head()

,id,date,trading_type,platform_id
0,11,2019-10-04,daily,1
1,12,2019-10-03,daily,2
2,13,2019-10-01,daily,2
3,14,2019-10-15,monthly,1
4,15,2019-10-15,monthly,2


In [14]:
charts_table = pd.read_sql('SELECT * FROM Chart_data', conn)
charts_table.head()

,id,time,lot_size,session_id,deal_id,price
0,0,11:34:47,2,12,0,-0.235500
1,1,11:32:37,2,13,1,-0.235500
2,2,11:52:45,13,13,2,-0.303677
3,3,11:53:07,25,13,3,-0.303677
4,4,12:00:43,124,13,4,-0.364279


In [200]:
def to_interval_ind(timestamp, session_breaks):
    return np.where(session_breaks >= timestamp)[0][0] - 1

def build_prices(period_prices, intervals=60, old_price=0):
    prices = []
    prev_price = old_price
    for i in range(intervals):
        prices.append(prev_price if i not in period_prices else period_prices[i])
        prev_price = prices[-1]
    return np.array(prices)

def prices_to_df(prices, times, platform, date):
    df = pd.DataFrame({
        'timestamp': times,
        'price': prices,
    })
    df['platform'] = platform
    df['date'] = date
    df['price'] -= df['price'].mean()
    df = df.sort_values(by=['timestamp'])
    return df

def load_sessions(sessions, charts, intervals=60):
    sessions = sessions.set_index('id')
    df = charts.join(sessions, on='session_id')
    df = df[df['trading_type'] == 'monthly']
    df = df.drop_duplicates(['deal_id'])
    df['deal_sum'] = df['price'] * df['lot_size']
    df.sort_values(by=['date', 'platform_id'])
    df = df.drop(columns=['trading_type', 'id', 'deal_id'])
    prices_sum = 0
    prices_count = 0
    result_sessions = {}
    s = df['session_id'].unique()
    for session in s:
        session_df = df[df['session_id'] == session]
        session_df = session_df.sort_values(by=['time'])
        platform = session_df['platform_id'].iloc[0]
        date = session_df['date'].iloc[0]
        min_time = session_df['time'].min()
        max_time = session_df['time'].max()
        session_start = min('11:00:00', min_time) if min_time < '12:00:00' else min(min_time, '12:00:00')
        session_end = max('12:00:00' if session_start == '11:00:00' else '13:00:00', max_time)
        session_period_breaks = pd.date_range(f'{date} {session_start}', f'{date} {session_end}', periods=intervals+1)
        session_df['timestamp'] = session_df['time'].apply(lambda t: f'{date} {t}')
        session_df['session_period'] = session_df['timestamp'].apply(lambda t: 
                                                                     to_interval_ind(t, session_period_breaks))
        periods = session_df.groupby('session_period')
        periods_prices = periods['deal_sum'].sum() / periods['lot_size'].sum()
        
        old_price = 0. if prices_count == 0 else 1. * prices_sum / prices_count
        prices = build_prices({i : p for i, p in zip(periods_prices.index, periods_prices.to_numpy())}, 
                              intervals,
                              old_price)
        prices_sum += session_df['deal_sum'].sum()
        prices_count += session_df['lot_size'].sum()
        result_sessions[(session, 'interpolated')] = prices_to_df(prices, session_period_breaks[:-1], platform, date)
        result_sessions[(session, 'exact')] = prices_to_df(session_df['price'], session_df['timestamp'], platform, date)
    return result_sessions, s

In [ ]:
def kmeans(n_clusters, distance, n_iters=1000)

In [201]:
def build_simple_plot(session_data):
    plt.clf()
    plt.plot(session_data.index, session_data['price'])
    plt.show()